# Importing necessary libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from tqdm import tqdm

# Mounting Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# Setting up processor

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# Loading and preprocessing the train data

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Applied ML HW 5/train.csv", encoding='unicode-escape')

train_labels = train['sentiment']
train_texts = train['text']

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

texts_list = []
for x in train_texts:
  texts_list.append(str(x))

train_encodings = tokenizer(texts_list, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
train_labels

array([1, 0, 0, ..., 2, 2, 1])

In [9]:
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(train_labels.tolist()))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 5)

# Training the model

In [13]:
for epoch in range(6):
    model.train()
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

100%|██████████| 3436/3436 [09:31<00:00,  6.02it/s]


# Loading and preprocessing test data

In [14]:
test = pd.read_csv("/content/drive/MyDrive/Applied ML HW 5/test.csv", encoding='unicode-escape')

test_labels = test['sentiment']
test_texts = test['text']

texts_list = []
for x in test_texts:
  texts_list.append(str(x))

test_encodings = tokenizer(texts_list, truncation=True, padding=True)

In [18]:
le = LabelEncoder()
test_labels = le.fit_transform(test_labels)

test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                              torch.tensor(test_encodings['attention_mask']),
                              torch.tensor(test_labels.tolist()))

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Evaluating the model

In [20]:
model.eval()
test_labels = []
test_preds = []
for batch in tqdm(test_loader):
    input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
    test_preds.extend(preds)
    test_labels.extend(labels.detach().cpu().numpy())

print(classification_report(test_labels, test_preds))

100%|██████████| 602/602 [00:20<00:00, 28.85it/s]

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      1001
           1       0.39      0.73      0.51      1430
           2       0.84      0.80      0.82      1103
           3       0.00      0.00      0.00      1281

    accuracy                           0.57      4815
   macro avg       0.49      0.58      0.53      4815
weighted avg       0.46      0.57      0.50      4815




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
